<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/stable/SteinhaugTrainer-JoePenna_dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dreambooth
#### Colab implementation by David Bielejeski. Latest information on: https://github.com/JoePenna/Dreambooth-Stable-Diffusion
##### Before starting, make sure you have the appropriate Accelerator and GPU Type selected from the Runtime menu `Runtime > Change runtime type`.  A minimum of 24GB of VRAM is required so you should select the A100 GPU (40GB). Both the T4 and V100 have less than 20GB.

In [1]:
#@title Check GPU and VRAM available. (Optional)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
print("\n\033[92mIf the available VRAM is equal or more than 24GB, then you are good to go.\033[0m")

NVIDIA A100-SXM4-40GB, 40960 MiB, 40511 MiB

If the available VRAM is equal or more than 24GB, then you are good to go.


In [2]:
#@title 1. Clone & Download The Repo
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion

Cloning into 'Dreambooth-Stable-Diffusion'...
remote: Enumerating objects: 1432, done.
remote: Counting objects: 100% (557/557), done.
remote: Compressing objects: 100% (268/268), done.
remote: Total 1432 (delta 346), reused 463 (delta 288), pack-reused 875
Receiving objects: 100% (1432/1432), 17.18 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (834/834), done.
/content/Dreambooth-Stable-Diffusion


In [ ]:
#@title 2. Build The Environment
#@markdown You might get warnings about restarting the runtime. Do this from the Runtime menu and after restarting, resume from Cell.
!pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython

import os
os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.1 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.5/707.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.3.1
    Uninstalling torchmetrics-1.3.1:
      Successfully uninstalled torchmetrics-1.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:

In [1]:
#@title 3. Just to ensure you are in the right directory.
%cd Dreambooth-Stable-Diffusion

/content/Dreambooth-Stable-Diffusion


In [ ]:
#@title 4.1. Alt 1: Download the 1.5 SD model with the improved VAE
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")


✅ model.ckpt successfully downloaded


In [2]:
#@title 4.2 Alt 2: Download checkpoint file with improved VAE from drive
from google.colab import drive
drive.mount('/content/drive')
# Copy checkpoint file from drive, remember to manually move checkpoint file into Dreambooth-Stable-Diffusion folder
%mkdir /content/checkpoint
!rsync -r -v --size-only --progress /content/drive/MyDrive/models/checkpoints/SDv15/. /content/checkpoint

Mounted at /content/drive
sending incremental file list
model.ckpt
  4,265,327,726 100%   60.69MB/s    0:01:07 (xfr#1, to-chk=0/2)

sent 4,266,369,163 bytes  received 35 bytes  58,846,471.70 bytes/sec
total size is 4,265,327,726  speedup is 1.00


In [3]:
#@title 5. Download Regularization Images
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0 <br />
from IPython.display import clear_output
dataset="man_unsplash" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# remove temp folder now it is empty.
!rm -rf Stable-Diffusion-Regularization-Images-{dataset}

clear_output()
print("✅ \033[92mRegularization Images downloaded.\033[0m")

Cloning into 'Stable-Diffusion-Regularization-Images-man_unsplash'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (396/396), done.
remote: Total 397 (delta 0), reused 397 (delta 0), pack-reused 0
Receiving objects: 100% (397/397), 20.04 MiB | 16.09 MiB/s, done.
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00000.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00000.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00001.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00001.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00002.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00002.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00003.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00003.jpg'
renamed 'Stable-

NameError: name 'clear_output' is not defined

In [4]:
#@title 6. Training Images
#@markdown ## Upload your training images
#@markdown WARNING: Be sure to upload an even amount of images, otherwise the training inexplicably stops at 1500 steps. <br />
#@markdown - 2-3 full body
#@markdown - 3-5 upper body
#@markdown - 5-12 close-up on face  <br /> <br />
#@markdown The images should be as close as possible to the kind of images you’re trying to make (most of the time, that means no selfies).<br /><br />
#@markdown If you get an error during uploading, just manually drag your training images into the training_images folder.
from google.colab import files
from IPython.display import clear_output

# Create the directory
!rm -rf training_images
!mkdir -p training_images

# Upload the files
uploaded = files.upload()
for filename in uploaded.keys():
 updated_file_name = filename.replace(" ", "_")
 !mv "{filename}" "training_images/{updated_file_name}"
 clear_output()

# Tell the user what is going on
training_images_file_paths = !find training_images/*
if len(training_images_file_paths) == 0:
 print("❌ \033[91mno training images found. Please upload images to training_images.\033[0m")
else:
 print("✅ \033[92m" + str(len(training_images_file_paths)) + " training images found.\033[0m")



✅ 28 training images found.


In [5]:
#@title 7. Final Setup & Training

#@markdown This isn't used for training, just used to name the folders etc
project_name = "steinhaug" #@param {type:"string"}

#@markdown This is the unique token i.e. you can use a nonsensical word like zwx or your name.
token = "stg" #@param {type:"string"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "man" #@param ["man", "person", "woman"] {allow-input: true}

# MAX STEPS
#@markdown How many steps do you want to train for?
max_training_steps = 5005 #@param {type:"integer"}

#@markdown If you are training a person's face, set this to True
i_am_training_a_persons_face = True #@param {type:"boolean"}
flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

#@markdown Would you like to save a model every X steps? (Example: 250 would output a trained model at 250, 500, 750 steps, etc)
save_every_x_steps = 401 #@param {type:"integer"}


reg_data_root = "/content/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "/content/Dreambooth-Stable-Diffusion/training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

2024-02-29 09:17:59.132130: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 09:17:59.183094: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 09:17:59.183136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 09:17:59.184728: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 09:17:59.192482: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
#@title 8. Save model into google drive
#@markdown This is often much faster than a manual download.  it will also save you compute units. <br />
from google.colab import drive
drive.mount('/content/drive')

# copy all ckpt files to google drive root dir
!cp trained_models/*.ckpt /content/drive/MyDrive/models/steinhaug